In [1]:
from omop_constructs.alchemy.adapters import map_lookup_view
from omop_constructs.alchemy.adapters import make_and_map_lookup_view, make_source_lookup
from omop_alchemy.cdm.model.clinical import Condition_Occurrence, Measurement
import sqlalchemy as sa
from orm_loader.helpers import Base

In [2]:
# this factory form can be used when you want to create 
# a lookup to your source data to match directly - 
# in this example, we want to match the condition_status_source_value 
# to the med_id in our source data, so we can use that to link back to source
# while accommodating type directly for convenience

# it also tracks where you've stored the source ID in your schema...which exposes some non-standard usage...

condition_source_lookup = make_source_lookup(
    model=Condition_Occurrence,
    source_column=Condition_Occurrence.condition_status_source_value,
    source_label="med_id",
    cast_to=sa.Integer(),
    name="condition_source_lookup",  # optional but nice
)

Condition_Source = map_lookup_view(
    base=Base,
    subquery=condition_source_lookup,
    name="Condition_Source",
    pk_cols=["condition_occurrence_id"],
)

In [3]:
list(Condition_Source.__table__.columns)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<condition_source_lookup>, nullable=False),
 Column('condition_occurrence_id', Integer(), table=<condition_source_lookup>, primary_key=True, nullable=False),
 Column('condition_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<condition_source_lookup>, nullable=False),
 Column('condition_start_date', Date(), table=<condition_source_lookup>, nullable=False),
 Column('condition_end_date', Date(), table=<condition_source_lookup>),
 <sqlalchemy.sql.elements.ColumnClause at 0x134754590; med_id>]

In [4]:
# or in a single step...

Measurement_Source = make_and_map_lookup_view(
    base=Base,
    name="Measurement_Source",
    model=Measurement,
    source_column=Measurement.measurement_source_value,
    source_label="obd_id",
    cast_to=sa.String(),
    pk_cols=["measurement_id"],
)

In [5]:
list(Measurement_Source.__table__.columns)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<measurement_source>, nullable=False),
 Column('measurement_id', Integer(), table=<measurement_source>, primary_key=True, nullable=False),
 <sqlalchemy.sql.elements.ColumnClause at 0x134841e50; obd_id>]

In [6]:
from omop_semantics.runtime.default_valuesets import runtime
runtime.condition_modifiers.condition_modifier_values.laterality

35918306

In [7]:
from omop_constructs.alchemy.joins.condition_modifier_joins import staging_subqueries, make_condition_modifier_fanout

staging_mapped_views = {
    name: map_lookup_view(
        base=Base,
        subquery=subq,
        name=f"{name.title().replace('_', '')}Modifier",
        pk_cols=["modifier_of_event_id", f"{name}_concept_id"],
    )
    for name, subq in staging_subqueries.items()
}

In [8]:
list(staging_mapped_views['t_stage'].__table__.columns)

[Column('modifier_of_event_id', Integer(), table=<t_stage>),
 Column('t_stage_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<t_stage>, nullable=False),
 Column('t_stage_date', Date(), table=<t_stage>, nullable=False),
 Column('t_stage_value', Integer(), ForeignKey('concept.concept_id'), table=<t_stage>)]

In [9]:
staging_mapped_views

{'t_stage': omop_constructs.alchemy.adapters.TStageModifier,
 'n_stage': omop_constructs.alchemy.adapters.NStageModifier,
 'm_stage': omop_constructs.alchemy.adapters.MStageModifier,
 'group_stage': omop_constructs.alchemy.adapters.GroupStageModifier,
 'grade': omop_constructs.alchemy.adapters.GradeModifier,
 'laterality': omop_constructs.alchemy.adapters.LateralityModifier,
 'path_stage': omop_constructs.alchemy.adapters.PathStageModifier}

In [10]:
condition_modifier_query = make_condition_modifier_fanout(
    base_cls=Condition_Occurrence,
    modifiers=staging_mapped_views,
    name="Condition_Modifier_Fanout",
)

In [12]:
class Condition_With_Modifiers(Base):
    __table__ = condition_modifier_query
    __mapper_args__ = {
        "primary_key": [
            condition_modifier_query.c.cancer_diagnosis_id
        ]
    }

In [13]:
list(Condition_With_Modifiers.__table__.columns)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<Condition_Modifier_Fanout>, nullable=False),
 Column('cancer_diagnosis_id', Integer(), table=<Condition_Modifier_Fanout>, primary_key=True, nullable=False),
 Column('cancer_start_date', Date(), table=<Condition_Modifier_Fanout>, nullable=False),
 Column('t_stage_value', Integer(), ForeignKey('concept.concept_id'), table=<Condition_Modifier_Fanout>),
 Column('t_stage_date', Date(), table=<Condition_Modifier_Fanout>, nullable=False),
 Column('n_stage_value', Integer(), ForeignKey('concept.concept_id'), table=<Condition_Modifier_Fanout>),
 Column('n_stage_date', Date(), table=<Condition_Modifier_Fanout>, nullable=False),
 Column('m_stage_value', Integer(), ForeignKey('concept.concept_id'), table=<Condition_Modifier_Fanout>),
 Column('m_stage_date', Date(), table=<Condition_Modifier_Fanout>, nullable=False),
 Column('group_stage_value', Integer(), ForeignKey('concept.concept_id'), table=<Condition_Modifier_Fanout>),
 Co